In [1]:
from pyspark.sql import SparkSession
import os 
SUBMIT_ARGS = "--packages graphframes:graphframes:0.7.0-spark2.4-s_2.11 pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = SUBMIT_ARGS
from spark import *
from graphframes import *
from pyspark import SparkContext
from pyspark.sql.functions import concat, col, lit
from pyspark.sql import *
from graphframes.examples import Graphs
print('done')

done


In [11]:
path = "201508_trip_data.csv"
edges = sqlContext.read.format("csv").option("header", "true").load(path).withColumnRenamed("Start Terminal", "src").withColumnRenamed("End Terminal", "dst").withColumnRenamed("Trip ID", "Trip-ID").withColumnRenamed("Start Date", "Start-Date").withColumnRenamed("Start Station", "Start-Station").withColumnRenamed("End Date", "End-Date").withColumnRenamed("End Station", "End-Station").withColumnRenamed("Bike #", "Bike-#").withColumnRenamed("Subscriber Type", "Subscriber-Type").withColumnRenamed("Zip Code", "Zip-Code").dropDuplicates()

path2 = "201508_station_data.csv"
vertexs = sqlContext.read.format("csv").option("header", "true").load(path2).withColumnRenamed("station_id", "id").dropDuplicates()


vertexs = vertexs.withColumn('lat-long', concat(col('lat'), lit(' : '), col('long')))
vertexs = vertexs.drop('lat')
vertexs = vertexs.drop('long')


g = GraphFrame(vertexs, edges)


# Display the vertex and edge DataFrames
g.vertices.show()

+---+--------------------+---------+-------------+------------+--------------------+
| id|                name|dockcount|     landmark|installation|            lat-long|
+---+--------------------+---------+-------------+------------+--------------------+
| 38|       Park at Olive|       15|    Palo Alto|   8/14/2013|37.4256839 : -122...|
| 46|Washington at Kea...|       15|San Francisco|   8/19/2013|37.795425 : -122....|
| 37|Cowper at University|       11|    Palo Alto|   8/14/2013|37.448598 : -122....|
| 45|Commercial at Mon...|       15|San Francisco|   8/19/2013|37.794231 : -122....|
| 22|Redwood City Calt...|       25| Redwood City|   8/15/2013|37.486078 : -122....|
|  3|San Jose Civic Ce...|       15|     San Jose|    8/5/2013|37.330698 : -121....|
| 36|California Ave Ca...|       15|    Palo Alto|   8/14/2013|37.429082 : -122....|
| 77|   Market at Sansome|       27|San Francisco|   8/25/2013|37.789625 : -122....|
| 56|     Beale at Market|       19|San Francisco|   8/20/2013|37

In [8]:
results = g.triangleCount()
print('Done')

Done


In [9]:
results.select("id", "count").show()
print('done')

+---+-----+
| id|count|
+---+-----+
| 51|  561|
|  7|   81|
| 54|  561|
| 11|   90|
| 29|   33|
| 69|  561|
| 42|  561|
| 73|  561|
| 64|  561|
|  3|   63|
| 30|   10|
| 34|   26|
| 59|  561|
|  8|   61|
| 22|   15|
| 28|   33|
| 16|   73|
| 35|   30|
| 71|  561|
| 47|  561|
+---+-----+
only showing top 20 rows

done


In [15]:
g.vertices.write.parquet("DataFrames/Edges")
g.edges.write.parquet("DataFrames/Vertices")
print('done')

done


In [19]:
results = g.pageRank(resetProbability=.15, maxIter=2)
print("done")

done


In [20]:
results.vertices.select("id", "pagerank").show()

print("done")

+---+------------------+
| id|          pagerank|
+---+------------------+
| 25|1.6024018319203723|
| 51|0.8884101250751557|
|  7|1.2524128714420097|
| 54|0.7581303586233339|
| 82|0.6544377565574545|
| 11|0.7670739160662481|
| 29|0.8981630140053439|
| 69|2.5122927719471106|
| 42|0.7908426658945884|
| 73|0.5950194420362506|
| 64|0.6834267434803176|
| 58|0.3660116138002388|
|  3|0.4646438771479797|
| 30|0.6946623287268426|
| 34| 1.387382352210988|
| 59|0.4637564730280793|
| 33|0.5668289972843616|
|  8|0.5428632723802358|
| 48|0.7589545837820103|
| 22|1.5732765002256561|
+---+------------------+
only showing top 20 rows

done


In [21]:
results.edges.select("src", "dst", "weight").show()
print("Done")


+---+---+--------------------+
|src|dst|              weight|
+---+---+--------------------+
| 54| 42|9.337068160597573E-4|
| 54| 42|9.337068160597573E-4|
| 54| 42|9.337068160597573E-4|
| 54| 42|9.337068160597573E-4|
| 54| 42|9.337068160597573E-4|
| 54| 42|9.337068160597573E-4|
| 54| 42|9.337068160597573E-4|
| 54| 42|9.337068160597573E-4|
| 54| 42|9.337068160597573E-4|
| 54| 42|9.337068160597573E-4|
| 54| 42|9.337068160597573E-4|
| 54| 42|9.337068160597573E-4|
| 54| 42|9.337068160597573E-4|
| 54| 42|9.337068160597573E-4|
| 54| 42|9.337068160597573E-4|
| 54| 42|9.337068160597573E-4|
| 54| 42|9.337068160597573E-4|
| 54| 42|9.337068160597573E-4|
| 54| 42|9.337068160597573E-4|
| 54| 42|9.337068160597573E-4|
+---+---+--------------------+
only showing top 20 rows

Done


In [33]:
results = g.shortestPaths(landmarks=["4", "16","45"])
print("done")

done


In [35]:
results.show()

+---+--------------------+---------+-------------+------------+--------------------+-----------------+
| id|                name|dockcount|     landmark|installation|            lat-long|        distances|
+---+--------------------+---------+-------------+------------+--------------------+-----------------+
| 25|Stanford in Redwo...|       15| Redwood City|   8/12/2013|37.48537 : -122.2...|               []|
| 51|Embarcadero at Fo...|       19|San Francisco|   8/20/2013|37.791464 : -122....|        [45 -> 1]|
|  7|Paseo de San Antonio|       15|     San Jose|    8/7/2013|37.333798 : -121....|[16 -> 1, 4 -> 1]|
| 54|Embarcadero at Br...|       15|San Francisco|   8/20/2013|37.787152 : -122....|        [45 -> 1]|
| 82|Broadway St at Ba...|       15|San Francisco|   1/22/2014|37.798541 : -122....|        [45 -> 1]|
| 11|         MLK Library|       19|     San Jose|    8/6/2013|37.335885 : -121....|[4 -> 1, 16 -> 1]|
| 29|San Antonio Caltr...|       23|Mountain View|   8/15/2013|37.40694 :